In [1]:
import os
import cv2 as cv
import mediapipe as mp
import time
import math
import numpy as np

In [34]:
cap = cv.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands()
result = os.popen("osascript -e 'output volume of (get volume settings)'").read()
t0=0
temp_vol=int(result.strip())
temp_coord=int(np.interp(temp_vol, [0, 100], [102, 448]))
while True:
    success, img = cap.read()
    img=cv.flip(img,1)
    if not success: 
        break
    imgRGB=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    vol=temp_vol
    coord=temp_coord
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            h, w, c = img.shape
            x1, y1 = (int(hand_landmarks.landmark[4].x*w), int(hand_landmarks.landmark[4].y*h))
            x2, y2 = (int(hand_landmarks.landmark[8].x*w), int(hand_landmarks.landmark[8].y*h))
            dist=math.hypot(x2 - x1, y2 - y1)
            temp_vol=np.interp(dist, [50, 350], [0, 100])
            vol=temp_vol
            os.system(f"osascript -e 'set volume output volume {temp_vol}'")
            cv.circle(img, (x1, y1), 5, (0, 255, 0), cv.FILLED)
            cv.circle(img, (x2, y2), 5, (0, 255, 0), cv.FILLED)
            cv.line(img, (x1, y1), (x2, y2), (255, 255, 255), 2)
            temp_coord=np.interp(vol, [0, 100], [102, 448])
            coord=temp_coord
    cv.putText(img,f"Volume level = {int(vol)} %", (100, 150), cv.FONT_HERSHEY_TRIPLEX, 1,(255, 200, 200),2) 
    cv.rectangle(img, (102, 177), (int(coord), 198), (255, 255, 255), -1)
    cv.rectangle(img, (100, 175), (450, 200), (0, 0, 255), 2) 
    t1=time.time()
    fps = 1/(t1 - t0)
    t0=t1
    cv.putText(img, str(int(fps))+ "fps", (100, 100), cv.FONT_HERSHEY_PLAIN, 3,(255, 0, 0),3)
    cv.imshow("Image", img)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()


I0000 00:00:1718691527.287921 1610065 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1718691527.301261 1644816 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718691527.306493 1644816 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
